In [ ]:
!pip install langchain langchain-openai pymongo bs4 openai tiktoken gradio requests lxml argparse unstructured boto3

In [13]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.document_loaders import DirectoryLoader
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import key_param

In [14]:
# 各種変数の設定
client = MongoClient(key_param.MONGO_URI)
dbName = "langchain_demo"
collectionName = "collection_of_text_blobs"
collection = client[dbName][collectionName]

In [15]:
# ディレクトリ内のテキストファイルをdataオブジェクトに読み込む
loader = DirectoryLoader( './sample_files', glob="./*.txt", show_progress=True)
data = loader.load()

100%|█| 3/3 [00:00<00:00


In [16]:
# ベクトル化を行うためのモデルを定義
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', api_key=key_param.openai_api_key)

In [17]:
# data に対するベクトルを生成して、langchain_demo.collection_of_text_blobsコレクションに追加する
vectorStore = MongoDBAtlasVectorSearch.from_documents( data, embeddings, collection=collection, index_name='langchain_demo' )

In [18]:
# 注：この先に進む前にAtlas UIでlangchain_demo.collection_of_text_blobsコレクションに対して
# 以下のVector Searchのインデックスを作成する

'''
{
  "fields": [{
    "path": "embedding",
    "numDimensions": 1536,
    "similarity": "cosine",
    "type": "vector"
  }]
}
'''

'\n{\n  "fields": [{\n    "path": "embedding",\n    "numDimensions": 1536,\n    "similarity": "cosine",\n    "type": "vector"\n  }]\n}\n'

In [21]:
def query_data(query):
    # OpenAIの埋め込みを使って質問をベクトルに変換し、
    # LangchainのvectorStoreを使って質問内容に関連のある文書をAtlas VectorSearchで検索し、
    # その文書の内容を考慮した回答を生成する

    # 質問文に最も関連がある文書1つをAtlas VectorSearchのsimilarity_searchで抽出
    docs = vectorStore.similarity_search(query, K=1)
    as_output = docs[0].page_content

    # 回答文生成に利用するOpenAIのLLMモデルを定義
    llm = OpenAI(model='gpt-3.5-turbo-instruct', api_key=key_param.openai_api_key, temperature=0)

    # OpenAIの文書生成モデルとAtlas VectorSearchを用いて、質問文に関連がある文書を踏まえた回答を生成
    retriever = vectorStore.as_retriever()
    qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)
    retriever_output = qa.invoke(query)

    # as_outout: 質問と関連が高い文書の内容
    # retriever_output: 上記の文書を踏まえて生成された回答内容
    return as_output, retriever_output['result']

In [22]:
# デモ画面の生成
with gr.Blocks(theme=Base(), title="Question Answering App using Vector Search + RAG") as demo:
    gr.Markdown(
        """
        # Question Answering App using Atlas Vector Search + RAG Architecture
        """)
    textbox = gr.Textbox(label="Enter your Question:")
    with gr.Row():
        button = gr.Button("Submit", variant="primary")
    with gr.Column():
        output1 = gr.Textbox(lines=1, max_lines=10, label="Output with just Atlas Vector Search (returns text field as is):")
        output2 = gr.Textbox(lines=1, max_lines=10, label="Output generated by chaining Atlas Vector Search to Langchain's RetrieverQA + OpenAI LLM:")

# Call query_data function upon clicking the Submit button

    button.click(query_data, textbox, outputs=[output1, output2])

demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [23]:
# 質問例
# 1. 8月16日にエラーは発生しましたか？もしそうなら、そのエラーの原因は何ですか？
# 2. アルフレッドはどのような質問をしましたか？日本語の箇条書きでまとめてください。
# 3. アルフレッドはブルースとどのような会話をしましたか？CSATはどのくらいでしたか？
# 4. yawとはなんですか?